# Notícias Indígenas Yanomami

*Notebook com o trabalho final da disciplina de Pensamento Computacional do Master em Jornalismo de Dados, Automação e Data Storytelling do Insper, disciplina ministrada pelo Álvaro Justen.*


### Objetivo

Acessar a URL que contém as notícias da [Terras Indígenas Yanomami](https://www.terrasindigenas.org.br/noticias/4016/TI/500/1), entrar em cada uma delas e verificar se cita o Rio Uraricoera, em caso afirmativo, salvar o link em um arquivo CSV.


### Resultado

Foi criado um código em que é possível extrair as notícias de qualquer uma das Terras Indígenas listadas no site, também é possível fazer a pesquisa que desejar dentro das notícias.

In [6]:
import csv

import requests
from bs4 import BeautifulSoup as bs

In [7]:
class NoticiasIndigenas:
    """ A classe extrai notícias de qualquer uma das Terras Indígenas listadas no site terrasingenas.org.br, também pode
    ser usada para verificar qual das notícias contêm determminada palavra e gravar as informações em um arquivo CSV
    (Data, Título e Link).
    
    Deve-se passar a URL.
    Caso queira verificar se alguma notícia contém a palavra, deve-se definir a palavra de interesse.
    Para gerar o arquivo CSV é necessário o nome do arquivo.
    """

    
    def baixar_html(self, site):
        """ Função que baixa o html do site. """
        
        r = requests.get(site)
        self.html = bs(r.text, "html.parser")
        
    def noticias_site(self):
        """ A função extrai todas as notícias do site. """
        
        lista = self.html.find("ul", {"id": "list-noticias"})     # Lista de notíciais do site - ul é o marcador do site
        tag_a = lista.find_all("a")     # tags "a" onde estão as informações
        # Separa link + título + data
        self.noticias_ti = []     # Lista onde serão armazenadas as notícias do site
        for tag in tag_a:
            texto = tag.text     # Título das notícias e data
            titulo = texto.split("-")     # Separa o título e a data da notícia
            url = "https://terrasindigenas.org.br" + tag.attrs["href"]     # Link das notícias
            self.noticias_ti.append({     # Dicionário que armazena as informações do site
                "Data": titulo[0],
                "Título": titulo[1],
                "Link": url,
            })
       
    
    def noticias_especificas(self, palavra= ""):
        """ A função extrai apenas as notícias que contêm a palavra definida. """
        
        especificas = []     # Lista onde serão armazenadas as notícias de nosso interesse
        for noticia in self.noticias_ti:
            r = requests.get(noticia["Link"]).text     # Entra em cada link
            if palavra in r.lower():     # Verifica se tem a palavra que estamos procurando, se tiver, armazena na variável
                especificas.append(noticia)
                self.especificas = especificas
    
    
    def escreve_csv(self, nome_arquivo):
        """ A função escreve no csv os resultados encontrados. """
        
        arquivo = open(nome_arquivo, mode="w")
        escritor = csv.DictWriter(arquivo, fieldnames=["Data", "Título", "Link"])  
        escritor.writeheader()
        for linha in self.especificas:
            escritor.writerow(linha)
        arquivo.close()

In [8]:
site = "https://terrasindigenas.org.br/noticias/4016/TI/500/1"
palavra = "uraricoera"
nome_arquivo = "uraricoera_yanomami.csv"

extrator = NoticiasIndigenas()
extrator.baixar_html(site)
extrator.noticias_site()
extrator.noticias_especificas(palavra)
extrator.escreve_csv(nome_arquivo)